<a href="https://colab.research.google.com/github/LC1332/Prophet-Andrew-Ng/blob/main/langchain/%E6%9D%8E%E9%B2%81%E9%B2%81%E5%AD%A6LangChain_25_Kor%E4%BF%A1%E6%81%AF%E6%8A%BD%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 李鲁鲁学LangChain 25 Kor信息抽取

这是李鲁鲁的学习笔记，对Sam Witteveen的油管课程的记录

增加了李鲁鲁学习时候的吐槽

因为一个独立的md文件导入知乎会更方便，所以这个课程的笔记会记录在md中

比如这节课在

https://github.com/LC1332/Prophet-Andrew-Ng/blob/main/langchain/%E6%9D%8E%E9%B2%81%E9%B2%81%E5%AD%A6LangChain%2025.md

欢迎访问[骆驼项目](https://github.com/LC1332/Luotuo-Chinese-LLM)主页和点赞

这篇笔记同时也是我们用来抽取Chat凉宫春日台本的原型代码

https://eyurtsev.github.io/kor/nested_objects.html

In [1]:
!pip -q install langchain openai google-search-results tiktoken
!pip -q install kor markdownify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00


In [2]:
!pip show langchain

Name: langchain
Version: 0.0.245
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: kor


In [3]:
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.llms import OpenAI
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)

In [ ]:
schema = Object(
    id="personal_info",
    description="Personal information about a given person.",
    attributes=[
        Text(
            id="first_name",
            description="The first name of the person",
            # examples=[("John Smith went to the store", "John")]
        ),
        Text(
            id="last_name",
            description="The last name of the person",
            # examples=[("John Smith went to the store", "Smith")],
        ),
        Number(
            id="age",
            description="The age of the person in years.",
            # examples=[("23 years old", "23"), ("I turned three on sunday", "3")]
        ),
    ],
    examples=[
        (
            "John Smith was 23 years old. He was very tall. He knew Jane Doe. She was 5 years old.",
            [
                {"first_name": "John", "last_name": "Smith", "age": 23},
                {"first_name": "Jane", "last_name": "Doe", "age": 5},
            ],
        )
    ],
    many=True,
)

In [ ]:
chain = create_extraction_chain(llm, schema)
print(chain.prompt.format_prompt(text="[user input]").to_string())

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

personal_info: Array<{ // Personal information about a given person.
 first_name: string // The first name of the person
 last_name: string // The last name of the person
 age: number // The age of the person in years.
}>
```


Please output the extracted information in CSV format in Excel dialect. Please use a | as the delimiter. 
 Do NOT add any clarifying information. Output MUST follow the schema above. Do NOT add any additional columns that do not appear in the schema.



Input: John Smith was 23 years old. He was very tall. He knew Jane Doe. She was 5 years old.
Output: first_name|last_name|age
John|Smith|23
Jane|Doe|5

Input: [user input]
Output:


In [ ]:
chain.run(
        "My name is Bob Alice and my phone number is (123)-444-9999. I found my true love one"
        " on a blue sunday. Her number was (333)1232832. Her name was Moana Sunrise and she was 10 years old."
)["data"]

In [5]:
import os

os.environ["OPENAI_API_KEY"] = "sk-Uf"

In [6]:
from langchain.llms import OpenAI
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)

In [ ]:
#@title 长的prompt

schema = Object(
    id="script",
    description="Adapted from the novel into script",
    attributes=[
        Text(
            id="role",
            description="The character who is speaking or performing an action",
        ),
        Text(
            id="dialogue",
            description="The dialogue spoken by the characters in the sentence",
        ),
        Text(
            id="action",
            description='''The actions performed by the characters in the text, A high-level summary of a character's behavior. action equals "对话" or "独白" if it's dialogue in sentence, equals other action if it's no dialogue''',
        )
    ],
    examples=[
        (
            '''韦小宝心中闪过一个念头：“我如得了这一千两赏银，我和妈娘儿俩可有得花了，鸡鸭鱼肉，赌钱玩乐，几年也花不光。”见茅十八仍是侧头瞧着自己，脸上神气颇有些古怪，韦小宝怒道：“你心里在想什么你猜我会去通风报信，领这赏银”茅十八道：“是啊，白花花的银子，谁又不爱”''',
            [
                {"role": "韦小宝", "dialogue": "我如得了这一千两赏银，我和妈娘儿俩可有得花了，鸡鸭鱼肉，赌钱玩乐，几年也花不光。","action":"独白"},
                {"role": "韦小宝", "dialogue": "你心里在想什么你猜我会去通风报信，领这赏银。","action":"对话"},
                {"role": "茅十八", "dialogue": "是啊，白花花的银子，谁又不爱。","action":"对话"},
            ],
        ),
        (
            '''澄观一直站在禅房门口等候。他内力深厚，韦小宝和那女郎的对答，虽微声细语，亦无不入耳，只觉这位师叔“劝说”女施主的言语，委实高深莫测，什么老公、老婆、孙子、爷爷，似乎均与武功无关，正自感佩赞叹，听得他问起解穴之法，忙道：“这位女施主被封的是大包穴，乃属足太阴脾经，师叔替她在腿上箕门、血海两处穴道推血过宫，即可解开。”''',
            [
                {"role": "澄观", "dialogue": "","action":"站在禅房门口听韦小宝和女郎对话，听到韦小宝询问解穴方法"},
                {"role": "澄观", "dialogue": "这位女施主被封的是大包穴，乃属足太阴脾经，师叔替她在腿上箕门、血海两处穴道推血过宫，即可解开。","action":"对话"},
            ],
        )
    ],
    many=True,
)

In [7]:
#@title 短的prompt

schema = Object(
    id="script",
    description="Adapted from the novel into script",
    attributes=[
        Text(
            id="role",
            description="The character who is speaking or performing an action",
        ),
        Text(
            id="dialogue",
            description="The dialogue spoken by the characters in the sentence",
        ),
        Text(
            id="action",
            description='''The actions performed by the characters in the text, A high-level summary of a character's behavior. action equals "对话" or "独白" if it's dialogue in sentence, equals other action if it's no dialogue''',
        )
    ],
    examples=[
        (
            '''韦小宝心中闪过一个念头：“我如得了这一千两，我和妈娘儿俩可有得花了” 见茅十八仍是侧头瞧着自己，韦小宝怒道：“你肯定猜我会去通风报信，领这赏银”茅十八道：“是啊，银子谁不爱”''',
            [
                {"role": "韦小宝", "dialogue": "我如得了这一千两赏银，我和妈娘儿俩可有得花了","action":"独白"},
                {"role": "韦小宝", "dialogue": "你肯定猜我会去通风报信，领这赏银","action":"对话"},
                {"role": "茅十八", "dialogue": "是啊，银子谁不爱","action":"对话"},
            ],
        ),
        (
            '''澄观一直站在禅房门口等候。韦小宝和那女郎的对答，虽微声细语，亦无不入耳，只觉这位师叔“劝说”女施主的言语，委实高深莫测，正自感佩赞叹，听得他问起解穴之法，忙道：“这位女施主被封的是大包穴，师叔替她解开即可”''',
            [
                {"role": "澄观", "dialogue": "","action":"站在禅房门口听韦小宝和女郎对话，听到韦小宝询问解穴方法"},
                {"role": "澄观", "dialogue": "这位女施主被封的是大包穴，师叔替她解开即可","action":"对话"},
            ],
        )
    ],
    many=True,
)

In [8]:
chain = create_extraction_chain(llm, schema)
print(chain.prompt.format_prompt(text="[user input]").to_string())

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

script: Array<{ // Adapted from the novel into script
 role: string // The character who is speaking or performing an action
 dialogue: string // The dialogue spoken by the characters in the sentence
 action: string // The actions performed by the characters in the text, A high-level summary of a character's behavior. action equals "对话" or "独白" if it's dialogue in sentence, equals other action if it's no dialogue
}>
```


Please output the extracted information in CSV format in Excel dialect. Please use a | as the delimiter. 
 Do NOT add any clarifying information. Output MUST follow the schema above. Do NOT add any additional columns that do not appear in the schema.



Input: 韦小宝心中闪过一个念头：“我如得了这一千两，我和妈娘儿俩可有得

In [9]:
response = chain.run(
        """黄龙使殷锦忙道：“夫人高见。取经之事，想来和福份大小，干系极大。黑龙使也不是不努力。不肯替教主立功，可是始终阻难重重，多半是福气不够，因此宝经难以到手。”洪夫人微笑道：“依你之见，谁的福份够呢”殷锦道：“本教福气最大的，自然是教主他老人家，其次是夫人。不过总不能劳动两位大驾亲自出马。更其次福份最大的，首推白龙使。他识得碣文，又立下大功，印堂隐隐透出红光，福份之大，教主属下无人能出其右。”

    教主捻须微笑，道：“但他小小孩童，能担当这件大任么”

    白龙使一职，在神龙教虽然甚尊，在韦小宝心里，却半点份量也没有，他既陷身岛上，只好随遇而安，瞧着闭月羞花的洪夫人。自是过瘾之极，但瞧得多了，如给教主发觉自己色迷迷的神色，难免有杀身之祸，还是尽速回北京为妙，听教主这么说，正是脱身的良机，便道：“教主，夫人，承蒙提拔，属下十分感激，我本事是没有的，但托了两位大福气，混进皇宫中去偷这四部宝经，倒也有成功的指望。”洪教主点了点头。洪夫人喜道：“你肯自告奋勇，足见对教主忠心。我知你聪明伶俐，福份又大，恐怕正是上天派来给教主办成这件大事的。”"""
)["data"]

In [13]:
for chat in (response['script']):
    print(chat)

{'role': '黄龙使殷锦', 'dialogue': '夫人高见。取经之事，想来和福份大小，干系极大。黑龙使也不是不努力。不肯替教主立功，可是始终阻难重重，多半是福气不够，因此宝经难以到手。', 'action': '对话'}
{'role': '洪夫人', 'dialogue': '依你之见，谁的福份够呢', 'action': '对话'}
{'role': '殷锦', 'dialogue': '本教福气最大的，自然是教主他老人家，其次是夫人。不过总不能劳动两位大驾亲自出马。更其次福份最大的，首推白龙使。他识得碣文，又立下大功，印堂隐隐透出红光，福份之大，教主属下无人能出其右。', 'action': '对话'}
{'role': '教主', 'dialogue': '但他小小孩童，能担当这件大任么', 'action': '对话'}
{'role': '白龙使', 'dialogue': '教主，夫人，承蒙提拔，属下十分感激，我本事是没有的，但托了两位大福气，混进皇宫中去偷这四部宝经，倒也有成功的指望。', 'action': '对话'}
{'role': '洪教主', 'dialogue': '点了点头', 'action': '对话'}
{'role': '洪夫人', 'dialogue': '你肯自告奋勇，足见对教主忠心。我知你聪明伶俐，福份又大，恐怕正是上天派来给教主办成这件大事的。', 'action': '对话'}


我们来批量运行下这个代码

In [14]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


save_folder = "/content/drive/MyDrive/GPTData/weixiaobao_extract"

确认这个folder的存在性，如果不存在这个folder，则新建这个folder

In [15]:
import os

save_folder = "/content/drive/MyDrive/GPTData/weixiaobao_extract"

if not os.path.exists(save_folder):
    os.makedirs(save_folder)

In [16]:
!wget https://raw.githubusercontent.com/LC1332/Prophet-Andrew-Ng/main/langchain/%E9%B9%BF%E9%BC%8E%E8%AE%B0.txt

--2023-07-28 07:13:39--  https://raw.githubusercontent.com/LC1332/Prophet-Andrew-Ng/main/langchain/%E9%B9%BF%E9%BC%8E%E8%AE%B0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3701970 (3.5M) [text/plain]
Saving to: ‘鹿鼎记.txt’

鹿鼎记.txt          100%[===================>]   3.53M  --.-KB/s    in 0.09s   

2023-07-28 07:13:40 (40.3 MB/s) - ‘鹿鼎记.txt’ saved [3701970/3701970]



In [17]:
#read /content/鹿鼎记.txt into python string raw_text
with open('/content/鹿鼎记.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

根据\n将 raw_text的文本切开，存储到split_text中，用python实现

In [19]:
split_text = raw_text.split('\n')

In [20]:
print(len(split_text))

22951


In [21]:
!pip -q install tiktoken

In [22]:
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

split_text中存储着一个python的list of string

已知 len(enc.encode( my_str )) 会计算my_str的token长度

定义max_token_len = 1000

生成一个新的数组chunk_text

里面的每个元素分别是split_text的顺序拼接

保证每个chunk_text的长度不要超过max_token_len

In [27]:
max_token_len = 1000
chunk_text = []

curr_len = 0
curr_chunk = ''

for line in split_text:
  line_len = len(enc.encode( line ))

  if line_len > max_token_len:
    print('warning line_len = ', line_len)

  if curr_len + line_len <= max_token_len:
    curr_chunk += line
    curr_chunk += '\n'
    curr_len += line_len
    curr_len += 1
  else:
    chunk_text.append(curr_chunk)
    curr_chunk = line
    curr_len = line_len

if curr_chunk:
  chunk_text.append(curr_chunk)

In [28]:
print(len(chunk_text))

1876


In [31]:
print(chunk_text[123])

    韦小宝回到屋中，向海老公道：“公公，你的法子不管用，太也稀松平常。”海老公哼了一声，说道：“没出息，又打输了。”韦小宝道：“如果用我自己的法子，虽然不一定准赢，也不见得准输。可是你的法子太也脓包，人家也都会的，有什么希奇”海老公奇道：“他也知道这法子你试给我瞧瞧。”韦小宝心想：“你眼睛瞎了，试给你看看，难道你看得见么”突然心念一动：“不知他是真瞎还是假瞎，可得试他一试。”当即双肘向后一撞，道：“他这么一撞，只撞得我全身三千根骨头，根根都痛。”海老公叹了口气，道：“你说这么一撞，我又怎瞧得见”颤巍巍的站起身来，道：“你试着学他的样。”韦小宝心下暗喜：“老乌龟是真的瞎了。”背心向着他，挺肘缓缓向后撞去，道：“他用手肘这样撞我。”待得手肘碰到了海老公胸口，便不再使力。海老公嗯了一声，说道：“这是腋底锤，那也算不了什么。”韦小宝道：“还有这样。”拉住了海老公左手，放在自己右肩，说道：“他用力一甩，我身子便从他头顶飞了过去。”这一招其实是他甩倒小玄子的得意之作，故意倒转来说，要考一考海老公。海老公道：“这是羚羊挂角。”韦小宝道：“原来你早知道了。”跟着拉住他手臂，慢慢而后拗转。海老公道：“嗯，这是倒折梅中的第三手。还有什么”
    韦小宝道：“原来小玄子这些手法都有名堂，我跟他乱打乱扭，那些手段可也得有几个好听的名堂才成啊。我向他扑过去，这小子向旁闪开，却在我背上顺势一推，我就”海老公不等他说完，便问：“他推在你哪里”韦小宝道：“他一推我便摔得七荤八素，怎还记得推在哪里。”海老公道：“你记记看。是推在这里么”说着伸手按在他左肩背后。韦小宝道：“不是。”海老公道：“是这里么”按在他右肩背后。韦小宝仍道：“不是。”海老公连按了六七个部位，韦小宝都说不是。海老公伸掌按在他右腰肋骨之下，问道：“是这里么”说着轻轻一推。韦小宝一个踉跄，跌出几步，立时记起小玄子推他的正是这个所在，大声道：“是了，一点不错，正是这里。公公，你怎么知道”




In [32]:
response = chain.run( chunk_text[123] )["data"]

In [33]:
for chat in (response['script']):
    print(chat)

{'role': '韦小宝', 'dialogue': '公公，你的法子不管用，太也稀松平常', 'action': '对话'}
{'role': '海老公', 'dialogue': '没出息，又打输了', 'action': '对话'}
{'role': '韦小宝', 'dialogue': '如果用我自己的法子，虽然不一定准赢，也不见得准输。可是你的法子太也脓包，人家也都会的，有什么希奇', 'action': '对话'}
{'role': '海老公', 'dialogue': '他也知道这法子你试给我瞧瞧', 'action': '对话'}
{'role': '韦小宝', 'dialogue': '他这么一撞，只撞得我全身三千根骨头，根根都痛', 'action': '独白'}
{'role': '海老公', 'dialogue': '你说这么一撞，我又怎瞧得见', 'action': '对话'}
{'role': '韦小宝', 'dialogue': '你试着学他的样', 'action': '对话'}
{'role': '韦小宝', 'dialogue': '他用手肘这样撞我', 'action': '独白'}
{'role': '海老公', 'dialogue': '这是腋底锤，那也算不了什么', 'action': '对话'}
{'role': '韦小宝', 'dialogue': '还有这样', 'action': '对话'}
{'role': '韦小宝', 'dialogue': '他用力一甩，我身子便从他头顶飞了过去', 'action': '独白'}
{'role': '海老公', 'dialogue': '这是羚羊挂角', 'action': '对话'}
{'role': '韦小宝', 'dialogue': '原来你早知道了', 'action': '对话'}
{'role': '韦小宝', 'dialogue': '他一推我便摔得七荤八素，怎还记得推在哪里', 'action': '独白'}
{'role': '海老公', 'dialogue': '你记记看。是推在这里么', 'action': '对话'}
{'role': '韦小宝', 'dialogue': '不是', 'action': '独白'}
{'role': '海老公',

In [ ]:
for i in range(0, len(chunk_text)):


已知
for chat in (response['script']):
    print(chat)

的输出结果如下

{'role': '韦小宝', 'dialogue': '公公，你的法子不管用，太也稀松平常', 'action': '对话'}
{'role': '海老公', 'dialogue': '没出息，又打输了', 'action': '对话'}
{'role': '韦小宝', 'dialogue': '如果用我自己的法子，虽然不一定准赢，也不见得准输。可是你的法子太也脓包，人家也都会的，有什么希奇', 'action': '对话'}
{'role': '海老公', 'dialogue': '他也知道这法子你试给我瞧瞧', 'action': '对话'}
{'role': '韦小宝', 'dialogue': '他这么一撞，只撞得我全身三千根骨头，根根都痛', 'action': '独白'}

定义save_name = 'output.jsonl'

将程序改写为把response['script']中的内容保存到save_name

In [35]:
# import json

# save_name = '/content/test_output.jsonl'

# with open(save_name, 'w', encoding='utf-8') as f:

#   for chat in response['script']:

#     json_str = json.dumps(chat, ensure_ascii=False)
#     f.write(json_str+"\n")


save_folder = "/content/drive/MyDrive/GPTData/weixiaobao_extract"

i从0到100循环

根据数字i 组织一个jsonl的保存文件名

如果文件名对应的文件已经存在，则跳过当前步骤

不然则运行

```python
with open(save_name, 'w', encoding='utf-8') as f:

  for chat in response['script']:
    
    json_str = json.dumps(chat, ensure_ascii=False)
    f.write(json_str+"\n")
```

进行写文件

整体的循环再增加一个进度条提示


In [40]:
import os
import json
from tqdm import tqdm

save_folder = "/content/drive/MyDrive/GPTData/weixiaobao_extract"

for i in tqdm(range(29,len(chunk_text))):

  save_name = os.path.join(save_folder, f"{i}.txt")

  if os.path.exists(save_name):
    continue

  response = chain.run( chunk_text[i] )["data"]

  with open(save_name, 'w', encoding='utf-8') as f:

    if 'script' not in response:
        print('Error: response does not contain key "script"')
    else:
        for chat in response['script']:
            json_str = json.dumps(chat, ensure_ascii=False)
            f.write(json_str+"\n")


#   break

  2%|▏         | 33/1847 [07:26<9:27:51, 18.78s/it]

Error: response does not contain key "script"


  6%|▌         | 105/1847 [34:02<10:06:24, 20.89s/it]

Error: response does not contain key "script"


  8%|▊         | 148/1847 [46:50<7:23:31, 15.66s/it]

...

100%|█████████▉| 1839/1847 [10:01:31<02:15, 16.94s/it]

Error: response does not contain key "script"


100%|█████████▉| 1840/1847 [10:01:46<01:55, 16.45s/it]

Error: response does not contain key "script"


100%|██████████| 1847/1847 [10:03:08<00:00, 19.59s/it]


In [39]:
print(i)

29


## 抽取哈利波特

In [1]:
!pip -q install langchain openai google-search-results tiktoken
!pip -q install kor markdownify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.1 MB/s eta 0:00:00


## Kor Basics

The basic workflow is the following:

1. Load the document
2. Clean up the document (optional)
3. Split the document into chunks
4. Define a schema for extraction
5. Extract from every chunk of text

In [ ]:
from typing import List, Optional

from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI

from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

import pandas as pd
from pydantic import BaseModel, Field, validator
from kor import extract_from_documents, from_pydantic, create_extraction_chain


from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter


## Simple examples

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
)

In [ ]:
schema = Object(
    id="personal_info",
    description="Personal information about a given person.",
    attributes=[
        Text(
            id="first_name",
            description="The first name of the person",
            examples=[("John Smith went to the store", "John")],
        ),
        Text(
            id="last_name",
            description="The last name of the person",
            examples=[("John Smith went to the store", "Smith")],
        ),
        Number(
            id="age",
            description="The age of the person in years.",
            examples=[("23 years old", "23"), ("I turned three on sunday", "3")],
        ),
    ],
    examples=[
        (
            "John Smith was 23 years old. He was very tall. He knew Jane Doe. She was 5 years old.",
            [
                {"first_name": "John", "last_name": "Smith", "age": 23},
                {"first_name": "Jane", "last_name": "Doe", "age": 5},
            ],
        )
    ],
    many=True,
)


chain = create_extraction_chain(llm, schema)

In [ ]:
print(chain.prompt.format_prompt(text="[user input]").to_string())

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

personal_info: Array<{ // Personal information about a given person.
 first_name: string // The first name of the person
 last_name: string // The last name of the person
 age: number // The age of the person in years.
}>
```


Please output the extracted information in CSV format in Excel dialect. Please use a | as the delimiter. 
 Do NOT add any clarifying information. Output MUST follow the schema above. Do NOT add any additional columns that do not appear in the schema.



Input: John Smith was 23 years old. He was very tall. He knew Jane Doe. She was 5 years old.
Output: first_name|last_name|age
John|Smith|23
Jane|Doe|5

Input: John Smith went to the store
Output: first_name|last_name|age
John||

Input: 

In [ ]:
result = chain.predict_and_parse(text="David Jones was 34 years old a long time ago.")

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


TypeError: ignored

## Nested Objects and JSON

In [ ]:
from_address = Object(
    id="from_address",
    description="Person moved away from this address",
    attributes=[
        Text(id="street"),
        Text(id="city"),
        Text(id="state"),
        Text(id="zipcode"),
        Text(id="country", description="A country in the world; e.g., France."),
    ],
    examples=[
        (
            "100 Main St, Boston, MA, 23232, USA",
            {
                "street": "100 Marlo St",
                "city": "Boston",
                "state": "MA",
                "zipcode": "23232",
                "country": "USA",
            },
        )
    ],
)

to_address = from_address.replace(
    id="to_address", description="Address to which the person is moving"
)

schema = Object(
    id="information",
    attributes=[
        Text(
            id="person_name",
            description="The full name of the person or partial name",
            examples=[("John Smith was here", "John Smith")],
        ),
        from_address,
        to_address,
    ],
    many=True,
)

### JSON encoding
To use nested objects, at least for now we have to swap to the JSON encoder.

In [ ]:
chain = create_extraction_chain(
    llm, schema, encoder_or_encoder_class="json", input_formatter=None
)

In [ ]:
print(chain.prompt.format_prompt(text="[user input]").to_string())

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

information: Array<{ // 
 person_name: string // The full name of the person or partial name
 to_address: { // Address to which the person is moving
  street: string // 
  city: string // 
  state: string // 
  zipcode: string // 
  country: string // A country in the world; e.g., France.
 }
 to_address: { // Address to which the person is moving
  street: string // 
  city: string // 
  state: string // 
  zipcode: string // 
  country: string // A country in the world; e.g., France.
 }
}>
```


Please output the extracted information in JSON format. Do not output anything except for the extracted information. Do not add any clarifying information. Do not add any fields that are not in the schema. If the tex

In [ ]:
chain.predict_and_parse(
    text="Alice Doe moved from New York to Boston, MA while Bob Smith did the opposite."
)["data"]

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


TypeError: ignored

## With Pydantic and validation

In [ ]:
!wget -q https://www.dropbox.com/s/gekyuep86zibhl1/conversation-025722052023.txt

#### Load the document

In [ ]:
def load_conversation(filename):

    with open(filename, 'r') as f:
        conversation = f.read()

    return conversation


In [ ]:
conversation = load_conversation('/content/conversation-025722052023.txt')

len(conversation)

9456

In [ ]:
conversation

'Food lover 2: Instruction: Please describe your first most unforgettable meal, including the location, ambiance, taste, and any unique experiences.\nInput: My first most unforgettable meal was at a restaurant called El Celler de Can Roca in Girona, Spain. The ambiance was elegant and modern, and the food was a creative and delicious 18-course tasting menu. One unique experience was when they brought out a dish that was inspired by the smells of the forest.\n\nFood lover 1: My response: That sounds amazing! The forest-inspired dish must have been a unique experience. My first most unforgettable meal was at a restaurant called Noma in Copenhagen, Denmark. The location was in an old warehouse by the waterfront, and the ambiance was rustic and cozy. The food was presented in a simple and natural way, with many of the ingredients sourced from the surrounding Nordic region. One of the most memorable dishes was a dessert made with fermented berries and ants, which added a surprising and deli

#### Split the text into docs

In [ ]:
doc = Document(page_content=conversation)

In [ ]:
split_docs = RecursiveCharacterTextSplitter().split_documents([doc])

#### Extract Restaurant Info


In [ ]:
llm = ChatOpenAI(
     model_name="gpt-3.5-turbo",
    temperature=0,
)

In [ ]:
class Restaurant(BaseModel):
    name: str = Field(
        description="The name of the restaurant",
    )
    location: Optional[str] = Field(
        description="City and or country, the place where the restaurant is",
    )
    style: Optional[str] = Field(
        description="The types of cusine that is served at the restaurant",
    )
    top_dish: Optional[str] = Field(
        description="The top dish that people love the most",
    )

    @validator("name")
    def name_must_not_be_empty(cls, v):
        if not v:
            raise ValueError("Name must not be empty")
        return v




In [ ]:
schema, extraction_validator = from_pydantic(
    Restaurant,
    description="Extract information about restaurants including their name, location, style and dishes.",
    examples=[
        (
            "My first fav meal was at a restaurant called Burnt Ends in Singapore.",
            {"name": "Burnt Ends", "location": "Singapore"},
        )
    ],
    many=True,
)

In [ ]:
chain = create_extraction_chain(
    llm,
    schema,
    encoder_or_encoder_class="csv",
    validator=extraction_validator,
    input_formatter="triple_quotes",
)

In [ ]:
print(chain.prompt.format_prompt(text="[user input]").to_string())

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

restaurant: Array<{ // Extract information about restaurants including their name, location, style and dishes.
 name: string // The name of the restaurant
 location: string // City and or country, the place where the restaurant is
 style: string // The types of cusine that is served at the restaurant
 top_dish: string // The top dish that people love the most
}>
```


Please output the extracted information in CSV format in Excel dialect. Please use a | as the delimiter. 
 Do NOT add any clarifying information. Output MUST follow the schema above. Do NOT add any additional columns that do not appear in the schema.



Input: """
My first fav meal was at a restaurant called Burnt Ends in Singapore.
"""
Output: 

In [ ]:
with get_openai_callback() as cb:
    document_extraction_results = await extract_from_documents(
        chain, split_docs, max_concurrency=5, use_uid=False, return_exceptions=True
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")

Total Tokens: 2745
Prompt Tokens: 2583
Completion Tokens: 162
Successful Requests: 3
Total Cost (USD): $0.00549


In [ ]:
document_extraction_results

[{'uid': '0',
  'source_uid': '0',
  'data': {'restaurant': [{'name': 'El Celler de Can Roca',
     'location': 'Girona, Spain',
     'style': '',
     'top_dish': 'forest-inspired dish'},
    {'name': 'Noma',
     'location': 'Copenhagen, Denmark',
     'style': 'Nordic cuisine',
     'top_dish': 'fermented berries and ants dessert'},
    {'name': 'La Cava del Tequila',
     'location': 'Mexico City, Mexico',
     'style': 'Mexican',
     'top_dish': 'mole'},
    {'name': 'Gaggan',
     'location': 'Bangkok, Thailand',
     'style': 'modern Indian cuisine',
     'top_dish': 'Lick It Up course'},
    {'name': 'Osteria Francescana',
     'location': 'Modena, Italy',
     'style': 'modern Italian cuisine',
     'top_dish': 'Oops! I Dropped the Lemon Tart'}]},
  'raw': 'name|location|style|top_dish\nEl Celler de Can Roca|Girona, Spain||forest-inspired dish\nNoma|Copenhagen, Denmark|Nordic cuisine|fermented berries and ants dessert\nLa Cava del Tequila|Mexico City, Mexico|Mexican|mole\nGag

#### Let's put it in a human readable format

In [ ]:
import json

def extract_restaurant_info(json_data):
    for record in json_data:
        restaurant_list = record.get('data', {}).get('restaurant', [])
        for restaurant in restaurant_list:
            name = restaurant.get('name', '')
            location = restaurant.get('location', '')
            style = restaurant.get('style', '')
            top_dish = restaurant.get('top_dish', '')

            # If style is not specified, we'll just say "Cuisine not specified"
            style = style if style else 'Cuisine not specified'

            print(f'Restaurant Name: {name}\nLocation: {location}\nStyle: {style}\nTop Dish: {top_dish}\n')



In [ ]:

extract_restaurant_info(document_extraction_results)

Restaurant Name: El Celler de Can Roca
Location: Girona, Spain
Style: Cuisine not specified
Top Dish: forest-inspired dish

Restaurant Name: Noma
Location: Copenhagen, Denmark
Style: Nordic cuisine
Top Dish: fermented berries and ants dessert

Restaurant Name: La Cava del Tequila
Location: Mexico City, Mexico
Style: Mexican
Top Dish: mole

Restaurant Name: Gaggan
Location: Bangkok, Thailand
Style: modern Indian cuisine
Top Dish: Lick It Up course

Restaurant Name: Osteria Francescana
Location: Modena, Italy
Style: modern Italian cuisine
Top Dish: Oops! I Dropped the Lemon Tart

Restaurant Name: Attica
Location: Melbourne, Australia
Style: Australian cuisine
Top Dish: Potato cooked in the earth it was grown



#### Lets put it in a structured DataFrame

In [ ]:
import pandas as pd

def generate_dataframe(json_data):
    # Prepare an empty list to store all restaurant data
    data = []

    for record in json_data:
        restaurant_list = record.get('data', {}).get('restaurant', [])
        for restaurant in restaurant_list:
            # Get details for each restaurant and append it to data
            data.append([
                restaurant.get('name', ''),
                restaurant.get('location', ''),
                restaurant.get('style', '') if restaurant.get('style', '') else 'Cuisine not specified',
                restaurant.get('top_dish', '')
            ])

    # Convert the list into a DataFrame
    df = pd.DataFrame(data, columns=['Name', 'Location', 'Style', 'Top Dish'])

    return df

# Usage:
df = generate_dataframe(document_extraction_results)


In [ ]:
df

,Name,Location,Style,Top Dish
0,El Celler de Can Roca,"Girona, Spain",Cuisine not specified,forest-inspired dish
1,Noma,"Copenhagen, Denmark",Nordic cuisine,fermented berries and ants dessert
2,La Cava del Tequila,"Mexico City, Mexico",Mexican,mole
3,Gaggan,"Bangkok, Thailand",modern Indian cuisine,Lick It Up course
4,Osteria Francescana,"Modena, Italy",modern Italian cuisine,Oops! I Dropped the Lemon Tart
5,Attica,"Melbourne, Australia",Australian cuisine,Potato cooked in the earth it was grown


In [ ]:
schema, validator = from_pydantic(Restaurant)

In [ ]:
chain = create_extraction_chain(
    llm,
    schema,
    encoder_or_encoder_class="csv",
    validator=validator,
    input_formatter="triple_quotes",
)

In [ ]:
with get_openai_callback() as cb:
    document_extraction_results = await extract_from_documents(
        chain, split_docs, max_concurrency=5, use_uid=False, return_exceptions=True
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")

Total Tokens: 2666
Prompt Tokens: 2412
Completion Tokens: 254
Successful Requests: 3
Total Cost (USD): $0.005332


In [ ]:
document_extraction_results

[{'uid': '0',
  'source_uid': '0',
  'data': {'restaurant': [{'name': '-',
     'location': '-',
     'style': '-',
     'top_dish': '-'},
    {'name': 'El Celler de Can Roca',
     'location': 'Girona, Spain',
     'style': 'Creative and delicious',
     'top_dish': 'Forest-inspired dish'},
    {'name': 'Noma',
     'location': 'Copenhagen, Denmark',
     'style': 'Simple and natural Nordic cuisine',
     'top_dish': 'Dessert made with fermented berries and ants'},
    {'name': 'La Cava del Tequila',
     'location': 'Mexico City',
     'style': 'Authentic and flavorful regional specialties',
     'top_dish': 'Mole'},
    {'name': 'Gaggan',
     'location': 'Bangkok, Thailand',
     'style': 'Creative and playful Indian cuisine',
     'top_dish': 'Lick It Up course'},
    {'name': 'Osteria Francescana',
     'location': 'Modena, Italy',
     'style': 'Modern take on traditional Italian cuisine',
     'top_dish': 'Oops! I Dropped the Lemon Tart'}]},
  'raw': 'name|location|style|top_di

In [ ]:

extract_restaurant_info(document_extraction_results)

Restaurant Name: -
Location: -
Style: -
Top Dish: -

Restaurant Name: El Celler de Can Roca
Location: Girona, Spain
Style: Creative and delicious
Top Dish: Forest-inspired dish

Restaurant Name: Noma
Location: Copenhagen, Denmark
Style: Simple and natural Nordic cuisine
Top Dish: Dessert made with fermented berries and ants

Restaurant Name: La Cava del Tequila
Location: Mexico City
Style: Authentic and flavorful regional specialties
Top Dish: Mole

Restaurant Name: Gaggan
Location: Bangkok, Thailand
Style: Creative and playful Indian cuisine
Top Dish: Lick It Up course

Restaurant Name: Osteria Francescana
Location: Modena, Italy
Style: Modern take on traditional Italian cuisine
Top Dish: Oops! I Dropped the Lemon Tart

Restaurant Name: 
Location: ---
Style: ---
Top Dish: ---

Restaurant Name: 
Location: Melbourne, Australia
Style: Australian
Top Dish: Potato cooked in the earth it was grown

Restaurant Name: 
Location: N/A
Style: Mexican
Top Dish: N/A

Restaurant Name: 
Location: N/A

In [ ]:
extraction_chain = create_extraction_chain(llm, restaurant_schema)

In [ ]:
sections[0]

'Food lover 2: Instruction: Please describe your first most unforgettable meal, including the location, ambiance, taste, and any unique experiences.\nInput: My first most unforgettable meal was at a restaurant called El Celler de Can Roca in Girona, Spain. The ambiance was elegant and modern, and the food was a creative and delicious 18-course tasting menu. One unique experience was when they brought out a dish that was inspired by the smells of the forest.\n\nFood lover 1: My response: That sounds amazing! The forest-inspired dish must have been a unique experience. My first most unforgettable meal was at a restaurant called Noma in Copenhagen, Denmark. The location was in an old warehouse by the waterfront, and the ambiance was rustic and cozy. The food was presented in a simple and natural way, with many of the ingredients sourced from the surrounding Nordic region. One of the most memorable dishes was a dessert made with fermented berries and ants, which added a surprising and deli

In [ ]:
text = sections[0]

extracted = extraction_chain.predict_and_parse(text=(text))["data"]

print(extracted)

{'restaurant': [{'name': 'El Celler de Can Roca'}, {'name': 'location'}, {'name': 'Girona, Spain'}, {'name': 'ambiance'}, {'name': 'elegant and modern'}, {'name': 'taste'}, {'name': 'creative and delicious 18-course tasting menu'}, {'name': 'unique experience'}, {'name': 'forest-inspired dish'}, {'name': 'name'}, {'name': 'Noma'}, {'name': 'location'}, {'name': 'Copenhagen, Denmark'}, {'name': 'ambiance'}, {'name': 'rustic and cozy'}, {'name': 'taste'}, {'name': 'simple and natural'}, {'name': 'unique experience'}, {'name': 'dessert made with fermented berries and ants'}]}


In [ ]:
def split_conversation(filename, max_tokens=1024):
    """
    Load a conversation from a file and split it into sections of approximately 2048 tokens.

    Parameters:
    filename (str): The name of the file to read the conversation from.
    max_tokens (int): The maximum number of tokens per section.

    Returns:
    list: A list of strings, where each string is a section of the conversation.
    """
    with open(filename, 'r') as f:
        conversation = f.read()

    # Split the conversation into turns
    turns = conversation.split("\n\n")

    sections = []
    section = ""

    for turn in turns:
        # If adding the next turn would exceed the maximum number of tokens,
        # add the current section to the list and start a new section
        if len(section.split()) + len(turn.split()) > max_tokens:
            sections.append(section.strip())
            section = ""

        # Add the turn to the current section
        section += f"{turn}\n\n"

    # Add the last section to the list
    sections.append(section.strip())

    return sections



In [ ]:
restaurant_schema = Object(
    id="restaurant",
    description=(
        "People are talking about restaurants names and dishes as well as qualities of the restaturant"
    ),
    attributes=[
        Text(
            id="name",
            description="The name of the restaurant"
        )
    ],
    examples=[("We went for a quick bite at McDonalds",[{"name": "McDonalds"}]),
            ("I just love the steaks at Mortons",[{"name": "Mortons"}]),
            ("We already have a booking at The Eatery so can't goto Mortons",[{"name": "The Eatery"},{"name": "Mortons"}])
            ],
    many=True,
)

### with browsing


In [ ]:
# Kor!
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

# LangChain Models
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# Standard Helpers
import pandas as pd
import requests
import time
import json
from datetime import datetime

# Text Helpers
from bs4 import BeautifulSoup
from markdownify import markdownify as md

# For token counting
from langchain.callbacks import get_openai_callback

def printOutput(output):
    print(json.dumps(output,sort_keys=True, indent=3))

### Load the text file

## Prepare the model

In [ ]:
llm = ChatOpenAI(
     model_name="gpt-3.5-turbo",
    temperature=0,
    # max_tokens=2048,
)

In [ ]:
restaurant_schema = Object(
    id="restaurant",
    description=(
        "People are talking about restaurants and dishes as well as qualities of the restaturant"
    ),
    attributes=[
        Text(
            id="name",
            description="The name of the restaurant"
        )
    ],
    examples=[("We went for a quick bite at McDonalds",[{"name": "McDonalds"}]),
                      ("I just love the steaks at Mortons",[{"name": "Mortons"}]),
                      ("We already have a booking at The Eatery so can't goto Mortons",[{"name": "The Eatery"},{"name": "Mortons"}])
                      ],
    many=True,
)

In [ ]:
# restaurant_schema = Object(

#     id="restaurant",

#     # Natural language description about your object
#     description="Personal information about a person",

#     # Fields you'd like to capture from a piece of text about your object.
#     attributes=[
#         Text(
#             id="first_name",
#             description="The first name of a person.",
#         )
#     ],

#     # Examples help go a long way with telling the LLM what you need
#     examples=[
#         ("Alice and Bob are friends", [{"first_name": "Alice"}, {"first_name": "Bob"}])
#     ]
# )

In [ ]:
chain = create_extraction_chain(llm, restaurant_schema)

In [ ]:
sections[0]

'Food lover 2: Instruction: Please describe your first most unforgettable meal, including the location, ambiance, taste, and any unique experiences.\nInput: My first most unforgettable meal was at a restaurant called El Celler de Can Roca in Girona, Spain. The ambiance was elegant and modern, and the food was a creative and delicious 18-course tasting menu. One unique experience was when they brought out a dish that was inspired by the smells of the forest.\n\nFood lover 1: My response: That sounds amazing! The forest-inspired dish must have been a unique experience. My first most unforgettable meal was at a restaurant called Noma in Copenhagen, Denmark. The location was in an old warehouse by the waterfront, and the ambiance was rustic and cozy. The food was presented in a simple and natural way, with many of the ingredients sourced from the surrounding Nordic region. One of the most memorable dishes was a dessert made with fermented berries and ants, which added a surprising and deli

In [ ]:
text = sections[0]
output = chain.predict_and_parse(text=(text))["data"]

printOutput(output)

{
   "restaurant": [
      {
         "name": "La Cava del Tequila"
      }
   ]
}


In [ ]:
output = chain.predict_and_parse(text=("The dog went to the park"))["data"]
printOutput(output)

{
   "person": []
}


## Multiple Fields

In [ ]:
 ("I had the fresh pasta with cream", "fresh pasta with cream"),
        #                 ("for me the steak frites was a good choice on my diet","steak frites"),
        #                 ("The grilled octopus was so yummy","grilled octopus"),
        #                 ("I had to send the fish tacos back as they were raw","fish tacos"),
        #             ],
        #     many=True,
        # ),
    ],
    many=True,
)

In [ ]:
with get_openai_callback() as cb:
    result = chain.predict_and_parse(text=text)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")

Total Tokens: 1858
Prompt Tokens: 1847
Completion Tokens: 11
Successful Requests: 1
Total Cost (USD): $0.0037159999999999997
